# A Simple Book Recommendation System

###### Done by __Safae HAJJOUT (@Ariyes)__ and __Mounia BADDOU (@MTheCreator)__ for Algorithmics class of 2022 / 2023.
###### Mohammed VI Polytechnic University, School of Computer Science, 1st year.


### ______________________________

##### This is a walkthrough of our simple recommendation system implemented using Python 3.10 and primitive data structures such as Linked lists, Arrays and some Python structures (Lists, Dictionaries, ... ).

##### The goal of this recommender system is to try and give the most accurate recommendations based on multiple criterias: titles, ratings, genres, description and others. We tried to make use also of some libraries to ease the recommendation process that is related to natural language (NLP).

#### Now we will go through the purpose of every chunk of code. Good reading :) .

### _____________________________

#### __First import what is needed !__

In [26]:
# Here can be downloaded every needed library for our code!
#! pip install --upgrade click
#! pip install nltk

In [27]:
from usefulclasses import Node, LinkedList, Queue, heapsort
from errors import *
import json
import nltk
import random
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download("punkt")
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#### We will be using the __Porter Stemmer algorithm__ to facilitate the recommendation

In [28]:
stopwords_set = set(stopwords.words('english'))
ps = PorterStemmer()

#### _______________________________

#### We can now load our book and user related data

In [29]:
with open('books.json') as file:
    books = json.load(file)
file.close()
    
with open('users.json') as file:
    users = json.load(file)
file.close()

#### _________________________________

## __Book Class__

##### One of our main classes, contains every valuable information on every book in our dataset in the shape of attributes (author, desc, title, ... ).
##### This class also contains one method: __str__ which serves as a display function.
##### More informations in our docstrings!

In [30]:
class Book:
    def __init__(self, author: str, desc: str, genre: list, isbn: str, pages: int, rating: float, title: str, totalratings: int):
        """ 
        Initializes a Book object with the provided details.

            Args:
                author (str): The author of the book.
                desc (str): A short description of the content of the book.
                genre (list): A list of genres that the book belongs to.
                isbn (str): The ID of the book.
                pages (int): The number of pages in the book.
                rating (float): The overall rating of the book on a scale of 0 to 5.
                title (str): The title of the book.
                totalratings (int): The total number of ratings given to the book.

            Returns:
                None
        """
        verify_book(author= author, desc= desc, genre= genre, isbn= isbn, rating= rating, total_ratings= totalratings, title= title)
        self.author = author                    
        self.desc = desc                        
        self.genre = genre                      
        self.isbn = isbn                        
        self.pages = pages                      
        self.rating = rating
        self.title = title 
        self.totalratings = totalratings      
        self.rate = self.rating * self.totalratings 
        
    def __str__(self):
        """ A display method where the __str__ function was overridden to meet the needs of our recommendation system. """
        return self.title + " by " + self.author

##### In this following chunk of code, we tried to sort our database of Book objects in a Python list using a heapsort to ease our recommendation based on ratings. 
##### The return will be the index, which will represent the rank of the book.

In [31]:
def create_books_list():
    """
    Creates a list of Book objects from the provided books data.

    Returns:
        list: A list of Book objects sorted in non-increasing order based on their ratings and total ratings.

    """
    books_list = []
    for book in books:
        author, desc, genre, isbn, pages, rating, title, totalratings = list(book.values())
        obj = Book(author, desc, list({g.lower() for g in genre.split(',')}), isbn, pages, rating, title, totalratings)
        books_list.append(obj)

    heapsort(books_list)
    return books_list

In [32]:
books_list = create_books_list()
size = len(books_list)

##### In this chunk, we try to tokenize our database in order to ease the recommendation using NLTK.
##### This operation will help reduce the size of the words we can use to recommend. (Stemming and Lemmatization).  

In [33]:
def create_books_dict(books_list):
    """
    Creates a dictionary of books using their ISBN as keys and a set of tokenized words from the title, description, and genre as values.

    Args:
        books_list (list): A list of Book objects.

    Returns:
        dict: A dictionary where the keys are the ISBNs of the books and the values are sets of tokenized words from the title, description, and genre.

    """
    verify_word = lambda word: word.lower() not in stopwords_set and word.isalnum()

    tokenized_title = lambda book: {word.lower() for word in book.title.split(' ') if verify_word(word)}
    tokenized_desc = lambda book: {word.lower() for word in book.desc.split(' ') if verify_word(word)}

    return { book.isbn : tokenized_title(book) | tokenized_desc(book) | set(book.genre) for book in books_list }
    

In [34]:
books_dict = create_books_dict(books_list)

#### _______________________________

## __User Class__

##### Our second Class for this code, where we store every information related to the user, such as the id, name, library, wishlist and other.

##### Almost every recommendation will be done through the User class since it is easier and recommends to each and every user.

##### For the methods of this class, they will be explained in the docstrings that accompany the code.

In [35]:
class User:
    def __init__(self, id: int, name: str, fav_genres: LinkedList):
        """
        Initializes a User object with the provided attributes.

        Args:
            id (int): The ID of the user.
            name (str): The name of the user.
            fav_genres (LinkedList): A linked list containing the user's favorite genres.
            library (Queue): A queue representing the user's library of books.
            wishlist (Queue): A queue representing the user's wishlist of books.

        """
        verify_user(user_id= id, name= name, genres= fav_genres)
        
        self.id = id
        self.name = name
        self.fav_genres = fav_genres
        self.library = Queue()
        self.wishlist = Queue()
        self.community = LinkedList()

    def read_book(self, book: Book, rate: int= 0):
        """
        Rates a book in the user's library.

        Args:
            book (Book): The book to be rated.
            rate (int): The rating to assign to the book.

        Returns:
            None

        """
        if type(book) != Book:
            raise TypeError("Book must be of type Book Object")
        
        print(f"the book {str(book)} was added successfully to your library ... ")
        self.library.enqueue((book, rate))
        print(f"You have rated the book {str(book)}: {rate} ... ")

    def add_to_wishlist(self, book: Book):
        """
        Adds a book to the user's wishlist.

        Args:
            book (Book): The book to be added to the wishlist.

        Returns:
            None

        """
        if type(book) != Book:
            raise TypeError("Book must be of type Book Object")
        self.wishlist.enqueue(book)

    def search_author(self, author: str) -> LinkedList:
        """
        Searches for books by a specific author.

        Args:
            author (str): The author's name to search for.

        Returns:
            LinkedList: A linked list of books written by the specified author.

        """
        books = LinkedList()
        for book in books_list:
            if author in book.author:
                books.prepend(book)
        return books

    def search_book(self, title: str) -> LinkedList:
        """
        Searches for books with a specific title.

        Args:
            title (str): The title of the book to search for.

        Returns:
            LinkedList: A linked list of books with the specified title.

        """
        verify_title(title)
        
        books = LinkedList()
        for book in books_list:
            if title in book.title:
                books.prepend(book)
        return books

    def search_book_isbn(self, isbn: str) -> LinkedList:
        """
    Searches for a book with a specific ISBN.

    Args:
        isbn (str): The ISBN of the book to search for.

    Returns:
        Book Object: The book with the specified ISBN.

    """
        verify_isbn(isbn)
        
        for book in books_list:
            if isbn in book.isbn:
                return book

    def search_genre(self, genre: str) -> LinkedList:
        """
    Searches for books within a specific genre.

    Args:
        genre (str): The genre to search for.

    Returns:
        LinkedList: A linked list of books belonging to the specified genre.

    """
        verify_genre(genre)
        
        books = LinkedList()
        for book in books_list:
            if genre.lower() in book.genre:
                books.prepend(book)
        return books

    def search_keywords(self, keywords: str, lower = 0, higher = size) -> LinkedList:
        """
    Searches for books based on one or more keywords .

    Args:
        keywords (str): Variable number of keyword arguments separated by commas (,).
        lower (int): lower bound of search in the index, the higher it is the higher the book's rank
        higher (int): higher bound of search in the index

    Returns:
        LinkedList: The books corresponding to the keywords

    """
        verify_lower_higher(lower, higher)
        
        keyterms = [ps.stem(term) for term in keywords.split(',')]
        books = LinkedList()
        for i in range(lower, higher):
            book = books_list[i]
            tokens = [ps.stem(word) for word in books_dict[book.isbn]]
            score = sum([1 if lemma in keyterms else 0 for lemma in tokens])
            # book must have at least 6 keyterms in common
            if score >= 6:
                books.prepend(book.title)
        
        return books

    def _find_max_count(self, data: list) -> str:
        """
    Helper function to find the element with the maximum count in a list.

    Args:
        data (list): A list of elements.

    Returns:
        str: The element with the maximum count.

    """
        counts = dict()
        for elem in data:
            # set count to 0 if checked for the first time else add 1
            counts[elem] = counts.get(elem, 0) + 1

        max_count = 0
        for (elem, count) in counts.items():
            (max_name, max_count) = (elem, count) if max_count < count else (max_name, max_count)

        return max_name

    def most_checked_genre(self) -> str:
        """
    Searches for the genre that has been checked out the most in the user's library.

    Returns:
        str: The genre with the highest check count from the user.

    """
        genres = [genre for node in self.library for genre in node.data[0].genre]
        if genres != []:
            return self._find_max_count(genres)

    def most_checked_author(self) -> str:
        """
    Searches for the author who has been checked out the most in the user's library.

    Returns:
        str: The author with the highest check count.

    """
        # same approach as genre
        authors = [author for node in self.library for author in node.data[0].author.split(',')]
        if authors != []:
            return self._find_max_count(authors)

    def recommend_books_genre(self) -> dict:
        """
    Recommends books based on the user's favorite genres. (Popularity Based Recommendation)

    Returns:
        dict: A dictionary where the keys are the user's favorite genres and the values are linked lists of recommended books.

    """
        books = dict()
        for genre in self.fav_genres:
            count = 3
            books[genre.data] = LinkedList()
            # traverse books_list (heap) in reverse order to get values in decreasing order
            for i in range(size - 1, -1, -1):
                if genre.data.lower() in books_list[i].genre and count != 0:
                    books[genre.data].append(books_list[i])
                    count -= 1
        return books
    
    def recommend_books(self) -> LinkedList:
        """
    Recommends books based on the user's past liked books in their library. (Content Based Recommendation)

    Returns:
        LinkedList: A linked list containing linked list nodes, each holding books similar to the ones 
                    the user liked in a range from 20 under to 20 upper to keep them close in rating

    """
        recommended = LinkedList()
        # check only for books in library with a rate >= 4
        liked_books = [node.data[0] for node in self.library if node.data[1] >= 4]
        for book in liked_books:
            idx = books_list.index(book)
            higher, lower = min(idx + 20, size), max(idx - 20, 0)
            string = ','.join(list(books_dict[book.isbn]))
            recs = self.search_keywords(string, lower, higher)
            recommended.prepend(recs)
        return recommended
    
    def add_community(self, users: list) -> None:
        """
    Searches for users with similar tastes to the user's and adds them to the community attribute (Book club function).

    Returns:
        None

    """
        for user in users:
            if user.most_checked_genre() in self.most_checked_genre() or user.most_checked_author() in self.most_checked_author():
                self.community.prepend(user)
                print(f"{user.name} with id: {user.id} was added successfully to your Reader Community !")

    def recommend_other_users(self) -> LinkedList:
        """
    Recommends books based on the user's community. (Collaborative Filtering based Recommendation)

    Returns:
        LinkedList: A linked list containing lists, each holding books liked by the user's friends
                    that are not already in the user's library

    """
        no_doubles = lambda book, lib: all(book.data[0].title != elem.data[0].title for elem in lib)
        
        experiment = LinkedList()
        for friend in self.community:
            friend_books = [node.data[0] for node in friend.data.library if no_doubles(node, self.library) and node.data[1] >= 3]
            for book in friend_books:
                experiment.prepend(book)
        
        return experiment 

In [36]:
def create_users_list():
    """
    Creates a list of User objects from the provided users data.

    Returns:
        list: A list of User objects.

    """
    def _generate_linked_list(py_l, linked_l):
        # helper function to generate linked list from python list
        for elem in py_l:
            linked_l.prepend(elem)
        return linked_l
    
    users_list = []
    for user in users:
        user_id, name, genres = list(user.values())
        obj = User(user_id, name, _generate_linked_list(genres, LinkedList()))
        users_list.append(obj)

    return users_list

users = create_users_list()

# <b>Project Simulation:</b>

In [37]:
sample_users = users[9855:9876]
sample_dict = {user.name: str(user.fav_genres) for user in sample_users}

print(sample_dict)

{'Dakari Hodges': 'Soviet History, Anthologies.', 'Eve Boone': 'Serbian Literature, Sword and Sorcery.', 'Mauricio Cooper': 'Christian, Futuristic.', 'Serenity Keller': 'Nonfiction, 20th Century.', 'Nico Wyatt': 'Mathematics, Vegetarian.', 'Liberty Lindsey': 'Programming Languages, Victor Frankenstein.', 'Jayson Fisher': 'Presidents, Science.', 'Arya Newman': 'Media Tie In, Anti Racist.', 'Anderson Dodson': 'French Revolution, Activism.', 'Etta Palmer': 'Teen, Culture.', 'Theo Contreras': 'Materials Science, Field Guides.', 'Daniela Huffman': 'Pseudoscience, Latin American Literature.', 'Chris Mendez': 'Southern Gothic, Christianity.', 'Londyn Hoffman': 'Gender, Comic Book.', 'Steven Maynard': 'English Civil War, Utopia.', 'Carolyn Leonard': 'Danish, Mauritius.', 'Ricardo Hancock': 'Harlequin, Spy Thriller.', 'Katelyn Whitaker': 'Trivia, Lebanon.', 'Keith Hawkins': 'American Fiction, Liberia.', 'Ariel Washington': 'High Fantasy, Abuse.', 'Juan Randolph': 'Outdoors, Ghost Stories.'}


In [38]:
dict_users = {}
for user in sample_users:
    packed_books = user.recommend_books_genre()
    books = {genre: [book.data for book in packed_books[genre]] for genre in packed_books}
    dict_users[user.id] = books

#### We will try to demonstrate some of our key methods for one user, then we will generalize for the whole __sample_users__ list

In [39]:
#User number 9855 appears to like the Soviet History and Anthologies genres, let's add to their wishlist books from the soviet union genre
user0 = sample_users[0]

for book in dict_users[user0.id]['Soviet History']:
    user0.add_to_wishlist(book)

In [40]:
for book in user0.wishlist:
    user0.read_book(book.data, random.randint(0, 5))

the book Stalin by Robert Service was added successfully to your library ... 
You have rated the book Stalin by Robert Service: 1 ... 
the book The Bolshevik Revolution 1917-23, Vol 1 by Edward Hallett Carr was added successfully to your library ... 
You have rated the book The Bolshevik Revolution 1917-23, Vol 1 by Edward Hallett Carr: 2 ... 
the book The Bolshevik Revolution 1917-23 by Edward Hallett Carr was added successfully to your library ... 
You have rated the book The Bolshevik Revolution 1917-23 by Edward Hallett Carr: 0 ... 


#### Now for the __sample_users__

In [41]:
# first add books to each users' wishlist
for user in sample_users[1:]:
    books = [book for genre in dict_users[user.id] for book in dict_users[user.id][genre]]
    for book in books:
        user.add_to_wishlist(book)    

In [42]:
# read and rate books
for user in sample_users:
    print(user.name, ": ")
    for book in user.wishlist:
        user.read_book(book.data, random.randint(0, 5))
    print("\n")

Dakari Hodges : 
the book Stalin by Robert Service was added successfully to your library ... 
You have rated the book Stalin by Robert Service: 5 ... 
the book The Bolshevik Revolution 1917-23, Vol 1 by Edward Hallett Carr was added successfully to your library ... 
You have rated the book The Bolshevik Revolution 1917-23, Vol 1 by Edward Hallett Carr: 5 ... 
the book The Bolshevik Revolution 1917-23 by Edward Hallett Carr was added successfully to your library ... 
You have rated the book The Bolshevik Revolution 1917-23 by Edward Hallett Carr: 3 ... 
the book Stalin by Robert Service was added successfully to your library ... 
You have rated the book Stalin by Robert Service: 1 ... 
the book The Bolshevik Revolution 1917-23, Vol 1 by Edward Hallett Carr was added successfully to your library ... 
You have rated the book The Bolshevik Revolution 1917-23, Vol 1 by Edward Hallett Carr: 3 ... 
the book The Bolshevik Revolution 1917-23 by Edward Hallett Carr was added successfully to you

#### Let's try the <b>content based recommendation system </b>

In [43]:
number = random.randint(0, 21)

random_user = sample_users[number]

recs = random_user.recommend_books()

In [44]:
print(f"The user {random_user.name} with favorite genres: {random_user.fav_genres} will probably enjoy these books:\n {recs}")

The user Anderson Dodson with favorite genres: French Revolution, Activism. will probably enjoy these books:
 Across Many Mountains: A Tibetan Family's Epic Journey from Oppression to Freedom (A Memoir), The Lady Tasting Tea: How Statistics Revolutionized Science in the Twentieth Century, Walking With The Comrades, The Source Field Investigations: The Hidden Science and Lost Civilizations Behind the 2012 Prophecies, The Ends of the Earth: A Journey to the Frontiers of Anarchy, Money and Power: How Goldman Sachs Came to Rule the World, Dani's Story: A Journey from Neglect to Love., Pitch Anything: An Innovative Method for Presenting, Persuading, and Winning the Deal, Between, Night Reigns, Barnaby Rudge, The Absolute Sandman, Volume Two, You're Next, Rebel Dawn, La reina estrangulada, Semper Fi, Day, The Game of Kings, Address Unknown, The Impossible Dead, Ysabel, Abundance, Tar Beach, Eleanor Rigby, Green Hills of Africa, Lord of the Vampires, Sarah, Chew, Vol. 4: FlambÃ©, The Cloister

#### Now for the <b>collaborative filtering recommendation system </b>

In [45]:
random_user.most_checked_genre()

'cultural'

In [46]:
random_user.add_community(sample_users)

Dakari Hodges with id: 9855 was added successfully to your Reader Community !
Anderson Dodson with id: 9863 was added successfully to your Reader Community !


In [47]:
recs = random_user.recommend_other_users()
for book in recs:
    print(book)

Nightmares and Dreamscapes by Stephen King
Everything's Eventual by Stephen King
The Bolshevik Revolution 1917-23 by Edward Hallett Carr
The Bolshevik Revolution 1917-23, Vol 1 by Edward Hallett Carr
The Bolshevik Revolution 1917-23 by Edward Hallett Carr
The Bolshevik Revolution 1917-23, Vol 1 by Edward Hallett Carr
Stalin by Robert Service
